In [7]:
import pandas as pd
import pymysql
import warnings
warnings.filterwarnings('ignore')

In [8]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [9]:
def select_data(table):
    sql = f'''
    SELECT 
        table_name AS '表名',
        ROUND(data_length/1024/1024, 2) AS '数据大小(MB)',
        ROUND(index_length/1024/1024, 2) AS '索引大小(MB)',
        ROUND((data_length + index_length)/1024/1024, 2) AS '总大小(MB)'
    FROM information_schema.TABLES 
    WHERE table_schema = '{table}' 
    ORDER BY (data_length + index_length) DESC;
    '''
    df = query(sql)
    return df

In [10]:
business = select_data('db_digua_business')
credit = select_data('db_credit')
rent = select_data('db_rent')
with pd.ExcelWriter('F:/周报及每日任务/表数据量排名.xlsx', engine='xlsxwriter') as writer:
    business.to_excel(writer, sheet_name='business', index=False)
    credit.to_excel(writer, sheet_name='credit', index=False)
    rent.to_excel(writer, sheet_name='rent', index=False)